# Вальчевський Павло, ОІ-21сп, Варіант 2, ПР № 4

In [56]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Зчитування даних, які не мають пустих значень
ds = pd.read_csv("./variant_2_new.csv")

## Кодування категоріальних стовпців

In [58]:
# Категоріальні стовпці
cat_columns = ['Date Egg', 'Clutch Completion', 'studyName', 'Region', 'Island', 'Stage', 'Individual ID', 'Sex', 'Species', 'Comments']

# Кодування категоріальних стовпців
map_dicts = dict()
for column in cat_columns:
    ds[column] = ds[column].astype('category')
    map_dicts[column] = dict(zip(ds[column], ds[column].cat.codes))
    ds[column] = ds[column].cat.codes

# Вивід результату
display(ds.sample(15))

,studyName,Sample Number,Region,Island,Stage,Individual ID,Clutch Completion,Date Egg,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Body Mass (g),Sex,Delta 15 N (o/oo),Delta 13 C (o/oo),Comments,Species
102,2,102,0,0,0,83,1,14,37.700000,16.000000,183.000000,3075.000000,0,9.215100,-26.22530,2,0
27,0,27,0,0,0,15,1,5,40.500000,17.900000,187.000000,3200.000000,0,8.081380,-25.49448,0,0
238,0,238,0,0,0,66,1,34,46.200000,14.500000,209.000000,4800.000000,0,7.840570,-25.37899,0,2
175,0,175,0,1,0,155,0,36,50.600000,19.400000,193.000000,3800.000000,1,9.281530,-24.97134,0,1
352,0,352,0,1,0,175,1,27,46.963361,17.569085,196.000000,3513.549642,0,9.364930,-24.31198,0,1
350,1,350,0,1,0,117,1,29,45.254115,17.136908,188.339153,3579.613464,0,9.650610,-24.48153,0,1
129,2,129,0,2,0,129,1,20,44.100000,18.000000,210.000000,4000.000000,1,9.107020,-26.01363,0,0
239,0,239,0,0,0,67,1,34,48.700000,15.100000,222.000000,5350.000000,1,10.354383,-25.39587,0,2
38,0,38,0,1,0,32,0,7,37.600000,19.300000,181.000000,3300.000000,0,9.411310,-25.04169,0,0
9,0,9,0,1,0,102,1,45,42.000000,20.200000,190.000000,4250.000000,1,9.133620,-25.09368,3,0


## Вибір метрик для оцінювання
- Accuracy: Загальна точність моделі.
- Precision: Частка правильних позитивних передбачень від загальної кількості позитивних передбачень.
- Recall: Частка правильних позитивних передбачень від загальної кількості фактичних позитивних класів.
- F1-score: Гармонійне середнє між precision і recall, яке використовується для незбалансованих даних.

In [60]:
from sklearn.metrics import classification_report

# Функція виводу метрик 
def print_classification_metrics(y_true, y_pred, name=str):
    print('Звіт метрик для:', name)
    print(classification_report(y_true, y_pred))

## Підхід для роботи з незбалансованими класи

In [62]:
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.utils import resample

# Створення ціольової змінної
ds['Target'] = ds['Species'] # Вид пінгвіна

# Розділ даних на класи
ds_class_0 = ds[ds['Target'] == 0] # Adelie
ds_class_1 = ds[ds['Target'] == 1] # Chinstrap
ds_class_2 = ds[ds['Target'] == 2] # Gentoo

# Виконуємо downsampling (зменшення вибірки), щоб балансувати класи
min_class_size = min(len(ds_class_0), len(ds_class_1), len(ds_class_2))

ds_class_0_downsampled = resample(ds_class_0, replace=False, n_samples=min_class_size, random_state=442)
ds_class_1_downsampled = resample(ds_class_1, replace=False, n_samples=min_class_size, random_state=442)
ds_class_2_downsampled = resample(ds_class_2, replace=False, n_samples=min_class_size, random_state=442)

# Вивід кількості об'єктів для кожного класу
print("Вивід кількості об'єктів для кожного класу:")
print(ds['Target'].value_counts())

# Об'єднуємо збалансовані класи
ds_balanced = pd.concat([ds_class_0_downsampled, ds_class_1_downsampled, ds_class_2_downsampled])

# Вибір ознак та цільової змінної
target = ds_balanced['Target']
features = ds_balanced.drop(['Target', 'Species'], axis=1)

# Розділ даних на тренувальні та тестові
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=442)

Вивід кількості об'єктів для кожного класу:
Target
0    152
1    152
2    152
Name: count, dtype: int64


## Вибір гіперпараметрів та їх оптимізація
### RandomizedSearchCV
- це метод оптимізації гіперпараметрів, основна мета якого знайти найкращих гіперпараметрів у моделі використовуючи випадковий підбір значень з простору гіперпараметрів.
- Метод дії:
- - визначення простору гіперпараметрів (задає користувач)
  - випадковий вибір (випадковим чином обирається певна кількість комбінацій гіперпараметрів з доступних, кількість комбінацій задається n_iter)
  - крос валідація (оцінка моделі, шляхом розбиття даних на k-1 тренувальних і 1 тестувального)
  - Вибір найкращої комбінації (вибір комбінації здійснюється за оцінкою, наприклад, F1)
### Random Forest
- стійкість до перенавчання
- висока продуктивність
- універсальність (може також розв'язувати задачі з незбалансованими класами)

In [64]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

rf = RandomForestClassifier(random_state=42)

# Гіперпараметри
param_distributions_rf = {
    'n_estimators': [1, 2],  # Кількість дерев
    'max_depth': randint(1, 5),  # Максимальна глибина кожного дерева
    'min_samples_split': randint(2, 20),  # Мінімальна кількість зразків для поділу вузла
    'min_samples_leaf': randint(2, 20),  # Мінімальна кількість зразків у листі
    'max_features': [0.1]  # Кількість ознак у %, що використовуються при кожному розгалуженні
}

# Оптимізація гіперпараметрів
random_search_rf = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_distributions_rf,
    n_iter=100,  # Кількість ітерацій
    scoring='f1',  # Метрика оцінювання (F1 score)
    cv=5,  # Поділ даних на 5 частин (4 тренуються, 1 тестуються)
    n_jobs=-1,  # Використання всіх доступних ядер процесора
    random_state=42 
)

# Навчання моделі
random_search_rf.fit(X_train, y_train)

# Прогнозування на тестовому наборі
y_pred = random_search_rf.predict(X_test)

# Вивід метрик
print_classification_metrics(y_test, y_pred, "Random Forest")

Звіт метрик для: Random Forest
              precision    recall  f1-score   support

           0       0.76      0.83      0.79        23
           1       0.89      0.84      0.86        37
           2       0.97      0.97      0.97        32

    accuracy                           0.88        92
   macro avg       0.87      0.88      0.87        92
weighted avg       0.88      0.88      0.88        92



### Logistic Regression
- Гарна продуктивність на малій вибірці
- Простота (відображає вплив кожної ознаки через коефіцієнти)
- Підтримка регуляризації (контроль складності моделі та запобігання перенавчанню)

In [66]:
from sklearn.linear_model import LogisticRegression
from scipy.stats import uniform

log_reg = LogisticRegression(random_state=42)

# Гіперпараметри
param_distributions_log_reg = {
    'C': uniform(0.1, 5),  # Рівень регуляризації
    'penalty': ['l1', 'l2', 'none'],  # Тип регуляризації (долучення втрат коефіцієнтів та їх квадратів)
    'solver': ['liblinear', 'saga', 'lbfgs'],  # Алгоритм для розв'язання (лінійний, градієнтний спуск, обмеженої пам'яті)
    'max_iter': randint(300, 500),  # Кількість ітерацій
    'tol': uniform(1e-6, 1e-2)  # Допустима похибка
}

# Оптимізація гіперпараметрів
random_search_log_reg = RandomizedSearchCV(
    estimator=log_reg,
    param_distributions=param_distributions_log_reg,
    scoring='f1',  # Метрика оцінювання (F1 score)
    cv=5,  # Поділ даних на 5 частин (4 тренуються, 1 тестуються)
    n_jobs=-1,  # Використання всіх доступних ядер процесора
    random_state=400
)

# Навчання моделі
random_search_log_reg.fit(X_train, y_train)

# Прогнозування
y_pred_log_reg = random_search_log_reg.predict(X_test)

# Вивід метрик
print_classification_metrics(y_test, y_pred_log_reg, "Logistic Regression")

Звіт метрик для: Logistic Regression
              precision    recall  f1-score   support

           0       0.88      0.96      0.92        23
           1       0.91      0.84      0.87        37
           2       0.91      0.94      0.92        32

    accuracy                           0.90        92
   macro avg       0.90      0.91      0.90        92
weighted avg       0.90      0.90      0.90        92



### SVC
- Ефективність для складних задач класифікації
- Підтримка ядерних функцій (kernel - для лінійних (linear) і нелінійних зв'язків (rbf) між даними)
- Гарна продуктивність на великих об'ємах даних

In [68]:
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint
from sklearn.metrics import classification_report

svm_model = SVC(random_state=42)

# Гіперпараметри
param_distributions_svm = {
    'C': uniform(1, 10),  # Рівень регуляризації
    'kernel': ['linear', 'rbf'],  # Ядро SVM (лінійне, радіально базисна функція)
    'gamma': uniform(0.01, 1),  # Числові значення для gamma (вплив окремих зразків на результат для радіально базисної функції)
    'max_iter': randint(1, 100)  # Кількість ітерацій
}

# Оптимізація гіперпараметрів
random_search_svm = RandomizedSearchCV(
    estimator=svm_model,
    param_distributions=param_distributions_svm,
    n_iter=100,  # Кількість ітерацій
    cv=5,  # Поділ даних на 5 частин (4 тренуються, 1 тестуються)
    n_jobs=-1,  # Використання всіх доступних ядер процесора
    random_state=42
)

# Навчання моделі
random_search_svm.fit(X_train, y_train)

# Прогнозування
y_pred_svm = random_search_svm.predict(X_test)

# Вивід метрик
print_classification_metrics(y_test, y_pred_svm, "SVC")


Звіт метрик для: SVC
              precision    recall  f1-score   support

           0       0.88      1.00      0.94        23
           1       0.94      0.84      0.89        37
           2       0.91      0.94      0.92        32

    accuracy                           0.91        92
   macro avg       0.91      0.93      0.92        92
weighted avg       0.92      0.91      0.91        92



## Вивід найкращих гіперпараметрів

In [70]:
print("Найкращі гіперпараметри RandomForest:", random_search_rf.best_params_)
print("Найкращі гіперпараметри Logistic Regression:", random_search_log_reg.best_params_)
print("Найкращі гіперпараметри SVM:", random_search_svm.best_params_)

Найкращі гіперпараметри RandomForest: {'max_depth': 3, 'max_features': 0.1, 'min_samples_leaf': 16, 'min_samples_split': 12, 'n_estimators': 2}
Найкращі гіперпараметри Logistic Regression: {'C': 3.4436445441708736, 'max_iter': 362, 'penalty': 'l2', 'solver': 'liblinear', 'tol': 0.007365666409074808}
Найкращі гіперпараметри SVM: {'C': 1.2541912674409519, 'gamma': 0.11789142699330445, 'kernel': 'linear', 'max_iter': 96}


## Висновки

Оцінка результатів:
- Random Forest - добре працює для всіх класів (найменш для 0 - 76 %) - загальна точність 88 %.
- Logistic Regression - добре працює для всіх класів - загальна точність 90 % 
- SVC - добре працює для всіх класів - загальна точність 91 %

Для налаштування гіперпараметрів використовував випадковий набір (geeksforgeeks.com).